This tutorial aims to guide CP2K users in the simulation of resonant X-Ray processes in Real-Time Time-Dependent DFT. 
It is related to the article XXX -- have a look for more details. 

The isolated carbon-monoxide molecule is taken as an example and the target frequency is the Oxygen K-edge (1s -> Lumo). 
We will use a hybrid functional (PBEh, with 45% exact exchange) with a rather large basis set (PCSEG-2). 
The procedure defined in this tutorial should be usable for other systems, functional, basis sets, or frequencies. 

It is constructed as follows: 
- Some Python initialization 
- How to choose your input parameters 
- How to analyze the results 

In the CP2K website, https://www.cp2k.org/howto:rtp_field_xas, you should find the output file corresponding to the mentioned run so that you can perform the analysis part directly

Notations: 

RTP: Real Time Propagation
LR: Linear Reponse
MO: Molecular Orbital


# Initialization

In [ ]:
# Loading some important packages and define some constants for unit conversions
%reset -f
import importlib
import numpy as np
from decimal import Decimal

import matplotlib
import matplotlib.pyplot as plt

au_to_ev = 27.211324570273 # convert atomic units to electron volts (for frequencies)
au_to_Vm = 5.142206747E+11 # convert atomic units to V/m (for field)
au_to_D = (8.4783536255 / 3.335640952) # convert atomic units to Debye (for dipole)
au_to_fs = 0.02418884254 # convert atomic units to femtosecond (for time)
from_ev_to_nm_at_529 = 2.343746559546314 # used to convert frequency from ev to nm

L_xyz = ['x', 'y', 'z'] # will be used later on for plots

# This tutorial is done with a carbone-monoxide molecule. It has 14 electrons in the all-atom description
Ne = 14 

We will use a Python package developed by the CP2K community: cp2kRTPtools. You can find it at https://github.com/glb96/cp2kRTPtools. It helps the reading of output files on Python and provides a few analyses, in particular Fourier transform. 
Of course, you can use your functions or preferred packages to perform the following analysis or to open the CP2K output files.

Once the package installed, for instance using pip, we load it:

In [ ]:
# this open-source package is available at: 
import cp2kRTPtools
print(importlib.reload(cp2kRTPtools))

from cp2kRTPtools import open_output_files
from cp2kRTPtools import ft_utils

In [ ]:
# some usefull function used later on

def gaussian(L_t, T_0, sig):
    '''
    The definition of the Gaussian in this tutorial
    '''
    L_y = np.exp(-1/2*((L_t - T_0)/sig)**2)
    return(L_y)

def gaussian_cos(L_t, omega, phi, E, T_0, sig):
    '''
    The electric field shape: a fast cosinus with a slow varying gaussian envelope
    '''
    L_y = E*np.cos(omega*L_t + phi)*gaussian(L_t, T_0, sig)
    return(L_y)
    
def tf_gaussian_cos(L_omega, omega, phi, E, T_0, sigma):
    '''
    Return the Fourier transform of a cos X gaussian. Note that some prefactor may be missing depending on the definition of the Fourier transform.
    '''
    L_tf = np.zeros(len(L_omega), dtype=complex)
    L_tf = np.exp(-1/2*((sigma*(L_omega-omega))**2))*np.exp(1j*(L_omega-omega)*T_0)*np.exp(1j*phi)
    return(L_tf)
    
def compute_envelope_vec_pot(L_time, delta_t, E, T_0, sig):
    '''
    The envelope-like vector potential used in the Area Theorem. 
    Please note that we use a naive integration of the field because it is closer to what is done in the Real-Time Propagation scheme (compared to a triagular approch for instance)
    
    WARNING: 
    The time step, delta_t, is expected to be in femtosecond as the other time inputs: L_time, T_0 and sig. 
    The field amplitude should be in atomic units.
    The returned integrated field is in atomic units
    '''
    
    L_envelope = E*gaussian(L_time, T_0, sig)

    L_A = np.zeros(len(L_envelope))
    for ttt in range(1, len(L_envelope), 1):
        L_A[ttt] = L_A[ttt-1] + L_envelope[ttt]

    L_A = L_A*delta_t/au_to_fs
    return(L_envelope, L_A)

def area_theorem_popu(tau, L_A):
    '''
    Return the excited population for both spin from the 2-electrons tau coefficient.
    
    Please note that this formula is for the effective promoting coefficient:
    \tau =  \left| \tilde{F}(\omega_{res})  \times  \mathbf{\mu}^{res} \cdot \mathbf{F} \right|^2 
    
    Considering two electrons. Yet the area theorem using the sin function is defined for one electron.
    Hence, we define the rate for one electron and multiply the excited population by 2.
    '''
    tau_one_electron = tau/2
    L_rho_exc = 2*np.sin(np.sqrt(tau_one_electron)*L_A/2)**2 
    return(L_rho_exc)

def computing_tau_from_rho_exc(L_rho_exc, L_A):
    '''
    Reverse formula of area_theorem_popu to find tau
    '''
    L_rho_exc_one_electron = L_rho_exc/2
    tau_one_electron = (2/L_A*np.arcsin(np.sqrt(L_rho_exc_one_electron)))**2
    tau = tau_one_electron*2
    return(tau)
    

# How to define the Real Time parameters?

Many parameters should be defined in order to perform Real-Time Propagation (RTP) simulation under a time-dependent field. 

The most important and the ones that will be extensively discussed in this tutorial are related to the time-dependent field applied: its frequency, polarization, shape, and amplitude. 
The simulation time step and total simulation time are also discussed. 
Other parameters related to the methodology of RTP are not presented here. 


One can always define the field parameters using experimental settings. 
Yet, the experimental conditions are often very demanding from a numerical point of view as the pulse can be quite long. 
Hence, this first part presents a reasonable way to select your RTP parameters depending on your system. 
We will see how to define these parameters thanks to Linear response TDDFT calculation, for instance, provided by the XAS_TDP module of CP2K.

## Reminders of XAS_TDP: how to read the results for RTP

A tutorial about XAS_TDP is available in the HOWTOs of the cp2k.org website: https://www.cp2k.org/howto:xas_tdp

For Linear Response calculation, we will not focus on the input section, presented in the above-mentioned tutorial, but rather on the output understanding. 
In the Data folder, you can find the LR directory containing the input and results of the XAS_TDP calculations for the Oxygen atom of isolated carbone-monoxide. 

Here are the first lines of the .spectrum files:

Each line corresponds to an available excited state associated with a frequency, in electron volts, and an oscillator strength in atomic units. 
Please note that we have used the Keyword  'XYZ_DIPOLE .TRUE.' in the &XAS_TDP section to extend the oscillator strength along the x, y, and z directions. 

The oscillator strength in the length approach is defined to be: 
$$
f^{osc} = \frac{2}{3} \omega_{res} \left|<\text{GS}|\hat{\mathbf{r}}|\text{EXC}>\right|^2 = \frac{2}{3} \omega_{res} \left| \mathbf{\mu}^{res}\right|^2
$$
where $<\text{GS}|\hat{\mathbf{r}}|\text{EXC}> = \mathbf{\mu}^{res} $ is the transition dipole moment from the ground state to the excited state separated by an energetic gap corresponding to the frequency $\omega_{res}$. 

In a two-state model approach, this would correspond to the Hamiltonian: 

$$
\hat{H} = \begin{bmatrix}
E_{gs} &  - \mathbf{\mu}^{res} \cdot \mathbf{F}(t)  \\
- \mathbf{\mu}^{res} \cdot \mathbf{F}(t)  & E_{exc}  \\
\end{bmatrix}
$$

where $E_{exc} - E_{gs} = \hbar \omega_{res}$ and assuming that it is expressed in the $|\text{GS}>, |\text{EXC}>$ basis set. 

The excited states ($|\text{EXC}>$) for the first 2 states can be found in the .wfn format (for CP2K) or .cube format (for visualization) in the LR directory. 


In [ ]:
# Please note that the frequency has to be converted into atomic unit: 
print(r'At 528.98 ev, $\left| \mathbf{\mu}^{res}\right|^2$ = ', 3/2*0.044017/(528.982511/au_to_ev))
print(r'At 535.45 ev, $\left| \mathbf{\mu}^{res}\right|^2$ = ', 3/2*0.000833/(535.449601/au_to_ev))

According to the perturbative approach, the resonant frequency, $\omega_{res}$, are poles of the linear polarizability.
The oscillator strength is linked to the residue value of these poles and describes how easily/probable an electron is promoted under a monochromatic field at this exact frequency. 

Hence, in RTP with an explicit electromagnetic field at the resonant frequency, the oscillator strength should be understood as the rate at which the core electrons are promoted to the excited state. 
This rate depends on the field polarization $\mathbf{F}$: the ''effective promotting rate'' $\tau$ according to the perturbative approach is given by:

$$
\tau =  | \mathbf{\mu}^{res} \cdot \mathbf{F}|^2 =  \left( F_x \mathbf{\mu}^{res}_{x} \right)^2 + \left( F_y \mathbf{\mu}^{res}_{y} \right)^2 + \left( F_z \mathbf{\mu}^{res}_{z} \right)^2
$$

Where $\mathbf{F}$ is the electric field polarization and $\mathbf{\mu}^{res}_{i}$ is the transition dipole moment along the x, y, and z directions.
From this coefficient, we can predict the time-dependent excited state population thanks to the area theorem, see for instance  https://iopscience.iop.org/article/10.1088/2058-9565/aa9269/pdf.
The induced excited population, $\rho^{\text{exc}}$, is given by the the time-dependent field amplitude, $E_{env}$ and the effective coupling:

$$
\rho^{\text{exc}}(t) = \sin  \left( \frac{ | \mathbf{\mu}^{res} \cdot \mathbf{F}| \times A(t)}{2} \right)^2  \approx    \tau \left( \frac{A(t)}{2} \right)^2  \\
A(t) = \int_{- \infty}^t E_{env}(t') dt'
$$

Where the time-dependent electric field is assumed to be resonant ($\omega_{0} = \omega_{res}$), with a cos function, and with a slow varying envelope, $E_{\text{env}}(t)$:

$$
\textbf{F}(t) = \textbf{F} E_{\text{env}}(t) \cos \left( \omega_{0} t + \phi \right).
$$

The goal of this tutorial is not to go into theoretical details, despite its beauty, and we can only recommend having a look at the two-state model rabi oscillation for people unfamiliar with this framework. 
Here, we will use this formalism, but with the general formula for time-dependent field shape (hence not monochromatic waves). 
In short, the main approximation is that the envelope (or shape) of the field $E_{\text{env}}$ is slowly varying compared to the field oscillation $\omega_0 = \omega_{res}$.

### Polarization discussion

From LR-TDDFT we know the frequency of the field to apply: it corresponds to the electronic transition targetted.
The effective promotion rate involves the scalar product between the field polarization and the transition dipole moment. 
Therefore, knowing which field polarization to apply is as important as knowing the frequency of the transition: using a field polarization so that $\mathbf{\mu}^{res} \cdot \mathbf{F}$ is zero would promote no electrons to this state even if the field frequency matches the electronic energy gap. 

According to this LR calculation, without MP2-frequency correction, the Oxygen K-edge is 528.98 ev and is degenerate: two states have the same frequency and total oscillator strength. 
The Carbon-Oxygen bond is along the z direction: the x and y directions are thus perpendicular to the CO bond and are symmetry equivalent. 
Looking at the decomposition of these dipolar transitions along the x, y, and z directions, we can see that the oscillator decomposition has no component along the z direction: these excited states are triggered by an excitation along the perpendicular plane only. 
Since the x and y directions are equivalent for the CO molecule, it leads to two degenerate states. 
In this calculation, the first state is triggered majority along the x direction and the second along the y direction. 
This decomposition is somehow arbitrary because of the xy symmetry equivalence. 

To summarize: the Oxygen K-edge frequency is at 528.98 ev and can be triggered by an electromagnetic field polarized along the XY plane: if one sends a field polarized along the XY plane, the effective promotion rate will be:

$$
\tau = | \mathbf{\mu}^{res} \cdot \mathbf{F}|^2 =  \frac{3}{2 \omega_{res}} \times  0.044 \times \mathbf{F}_{xy}^2 = 3.4 \ 10^{-3} \times \mathbf{F}_{xy}^2
$$


The third available excited state at 535.45 ev is easier to understand: its transition dipole moment is along the z direction only. 
Hence, to excite this state, one should emit an electromagnetic field with a frequency of 535.45 ev and polarized along the z direction: 

$$
\tau = | \mathbf{\mu}^{res} \cdot \mathbf{F}|^2 = \frac{3}{2 \omega_{res}} \times 0.0008 \times \mathbf{F}_{z}^2 = 6.3 \ 10^{-5} \times \mathbf{F}_{z}^2
$$

We will neglect the next available states because they are too far away from the K-edge.  


We want to simulate in real-time the excitation of the Oxygen K-edge. 
According to the previous discussion based on the XAS_TDP results, this means that we have to send an electromagnetic field with frequency 528.98 ev and polarized in the xy plane. 

Usually, one wants to be as state-specific as possible: for instance by avoiding the excitation of the next available state at 535.45 ev. 
There is at least two way of achieving this goal: being **frequency selective** or **polarization selective**. 
Frequency selective means that the electromagnetic field applied to the system shows no frequency amplitude at 535.45 ev.
Polarization selective means that the electromagnetic field has no components along the z direction. 
For the carbon monoxide isolated molecule, it is easy to avoid the excitation at 535.45 ev using the polarization specificity since the one we target (in the xy plane) is orthogonal to the one we want to avoid (along the z-axis). 

Yet, in many cases, and especially in solvated systems, such situations do not exist because of the loss of spatial symmetry. 
Below is presented the XAS_TDP output for solvated cysteine molecule for its Sulfur K-edge: 


In [ ]:
# Please note that the frequency has to be converted into atomic unit: 
print(r'At 2438.34 ev, $\left| \mathbf{\mu}^{res}\right|^2$ = ', 3/2*0.005468/(2438.342456/au_to_ev))
print(r'At 535.45 ev, $\left| \mathbf{\mu}^{res}\right|^2$ = ', 3/2*0.002299/(2440.416927 /au_to_ev))

print(3/2/(2440.416927 /au_to_ev)*((0.902**2)*0.002299 + (0.431**2)*0.002165))

In this case, the edge and the second available states are at 2438.34 and 2440.42 ev respectively. 
The edge can be excited by a field along the xz plane and so the second state. 

Hence, one choice for the field polarization would be to maximize the overlap with the oscillator strength of the targetted state. 
For instance, one can use the same direction for the field polarization as the one of the transition dipole.
For the K-edge, the polarization would be: 

$$
\textbf{F} = \frac{\mathbf{\mu}^{res}}{|\mathbf{\mu}^{res}|} = 
\begin{bmatrix} 
0.902 \\ 
 0.0 \\ 
0.431
\end{bmatrix} 
$$

In this case, the effective coupling due solely to the polarization direction is for the edge: 

$$
\tau_{edge} =  |\mathbf{\mu}^{res} \cdot \mathbf{F} |^2 = |\mathbf{\mu}^{res}|^2 = \frac{3}{2 \omega_{res}} \times 0.00547 = 9.1 \ 10^{5}
$$

The effective coupling due solely to the polarization direction for the second state is: 
$$
\tau_{second} =   |\mathbf{\mu}^{res} \cdot \mathbf{F} |^2 =  \frac{3}{2 \omega_{res}} \left( 0.902^2 \times 0.002299 + 0.431^2 \times  0.002165 \right) = 3.8 \ 10^{-5}
$$

Another choice could be to minimize the effective rate of the state we do not want to excite.

In any case, there will still be some electronic excitation toward the second state when applying a field to promote electrons toward the edge because of their transition dipole directions. 
Hence, to reduce the promoting rate of the second state, one should also play with the shape of the electromagnetic field to avoid the frequency associated with the second electronic transition. 


### Frequency discussion

The electromagnetic field one applies in a Real-Time simulation is not monochromatic -- as in real life. 
Indeed, it has at least a beginning and an end due to the finite time simulated.
Its shape $E_{env}$ is often chosen to be Gaussian, as the one used in many experiments. 

The area theorem, which links the excited state population to the field shape, 

$$
\rho^{\text{exc}}(t) \approx \tau \left( \frac{A(t)}{2} \right)^2  \\
A(t) = \int_{- \infty}^t E_{env}(t') dt'
$$

should be understood for a field resonant with respect to the targetted transition: 

$$
F(t) = \textbf{F} E_{\text{env}}(t) \cos \left( \omega_{0} t + \phi \right)
$$

Where $\omega_0 = \omega_{res}$.

Yet, this time-dependent electric field has a width in the Fourier domain: it is the time production of an envelope and a cos, so in the Fourier space it is the convolution of the Fourier transform of the envelope with a delta Dirac centered at the $\omega_{exc}$ frequency. 
Hence, the width of the field in the Fourier space is given by the width of the envelope. 

As a general rule, the more spread in the time domain, the more sharp in the Fourier one: applying a field during a long period of time will lead to a very selective excitation (sharp Fourier space) while applying a very short pulse would tend to trigger several excitations. 



One can rewrite the area theorem with the Fourier transform of the field: 

$$
\rho^{\text{exc}}(t) \approx \tau \left( \frac{\tilde{F}(\omega_{res}) A(t)}{2} \right)^2  \\
\tilde{F}(\omega_{res}) = \left| \frac{\text{FT} \left[ F \right] (\omega_{res})}{\text{max} \left( \text{FT}\left[ F \right] \right)} \right| = \left| \frac{F(\omega_{res})}{\text{max} \left( F(\omega) \right)} \right|
$$

Where $\tilde{F}(\omega_{res})$ is the relative weight of the Fourier transform of the field at the targetted frequency $\omega_{res}$. 
Here we have assumed that the available excited states dynamics do not interact with themselves.

One can include this frequency effect by updating the effective promoting rate: 

$$
\rho^{\text{exc}}(t) \approx \tau(\omega_{res}) \left( \frac{A(t)}{2} \right)^2  \\
\tau(\omega_{res}) =  \left| \tilde{F}(\omega_{res})  \times  \mathbf{\mu}^{res} \cdot \mathbf{F} \right|^2 
$$

Here $\tau(\omega_{res})$ contains the two selectivities mentioned previously: the one related to the field polarization with respect to the transition dipole, $ \mathbf{\mu}^{res} \cdot \mathbf{F}$, and the one related to the matching of the field frequency with respect to the one of the transition targetted $\omega_{res}$, $ \tilde{F}(\omega_{res})$.

If there are several transitions available, the time-dependent electric field will promote electrons with the associated rate $\tau(\omega_{res})$ toward each available state. 

## Defining the RTP parameters

Now that we have reminded some key elements about Linear Response calculation and its link to the real-time properties, let us go into the details of the RTP parameters choice.


### Time step value: main frequency sampling

Usually, the time step is given by the fastest electronic dynamics in the system. 
Here, we are applying to the system a perturbation that triggers this dynamics: 1s-Lumo excitation. 
Hence, a reasonable value for the time step is related to this resonant frequency. 

According to LR-TDDFT calculation, we have for the Oxygen atom the K-edge at 529 ev with an oscillator strength of 0.0440 a.u. for carbon monoxide. 
This means that the field main frequency will be $\omega_0=\omega_{res}$ = 529 ev. 
Hence, the fastest frequency is 529 ev. 
To describe these dynamics, the time step should be short enough so that about 5-10 time steps describe one field period. 
If we note $n$ the number of time steps needed to describe one period, $n$ fulfills:

$$
\omega_0 \times n \times dt = 2 \pi 
$$

where $dt$ is the time step value.
For $n = 10$ and $\omega_0$  = 529 ev, we found $dt = 7.8$ attosecond.

One can always use an even shorter time step, but this leads to a more expensive calculation. 

In [ ]:
# Your parameters:
E = 1 # here it does not matter, please leave it to 1.
omega_ev = 529 #the frequency of the targetted state in ev
phi = 0*np.pi/180 # phase of the pulse. 0 means +cos, -90 means +sin.
N_ts_per_period = 10 # how many time step per period, try between 5-10. 


# Output
omega_fs = omega_ev/(au_to_ev*au_to_fs)
delta_t = 2*np.pi/(omega_fs*N_ts_per_period) #fs

print('The frequency for the fast oscillation in nm is:', (529/omega_ev)*from_ev_to_nm_at_529, ' in nm')
print('The time step value is:', delta_t, ' in fs')


### Field polarization: which state is targetted

We want to describe the absorption of carbon monoxide at the Oxygen K-edge. 
This means that we want the core electrons to be promoted to one specific state and not all the available ones. 
Thus, the time-dependent field should oscillate around the K-edge frequency and should be sharp enough in the frequency domain to prevent another type of excitation (for instance 1s -> Lumo+1).

According to LR-TDDFT calculation, the edge can be excited with a field in the xy plane while the next available frequency is at 535.5 ev and is excited by a field along the z-axis. 

Hence, applying a field in the xy plane, for instance along x, at 529 ev will promote electron toward the first state, and not toward the second available one since $\mathbf{\mu}^{ 535.5 \text{ev}} \cdot \mathbf{F}_{xy} = 0$.

Thus, the field polarization for this simulation will be along x.


### Width in time domain: how sharp in the frequency domain

For the sake of example, we will still assume that we need the frequency selectivity: we shall not excite the second available excited state at 535.5 ev.
This means that we would like to send a time-dependent electric field pulse with a frequency width of about 6~ev around the central frequency.
This way, the K-edge electronic transition should be triggered (at 529 ev) but not the next one (at 535.5 ev).
In other words: $\tilde{F}(529 \text{ev}) = 1 $ and  $\tilde{F}(535.5 \text{ev}) \approx 0 $.

The time-dependent field that we will be using is a cos function multiplied with a Gaussian:

$$
F(t) = E_{\text{env}}(t) \cos \left( \omega_0 t + \phi \right) = E \, \exp \left[ - \frac{1}{2} \left( \frac{T_0-t}{\sigma} \right)^2 \right]  \cos \left( \omega_0 t + \phi \right)
$$

Its Fourier transform is given by: 

$$
F(\omega)  \propto E \exp \left[ i (\omega-\omega_0) T_0 + \phi \right] \, \exp \left[ -\frac{1}{2} \sigma^2 \left( \omega - \omega_0 \right)^2 \right] 
$$

So the absolute value for the positive frequencies is given by:

$$
| F(\omega) | \propto E \, \exp \left[ -\frac{1}{2} \sigma^2 \left( \omega - \omega_0 \right)^2 \right]
$$

Hence, for a Gaussian pulse, the relative amplitude of the field in the frequency domain is given by a Gaussian centered at the resonant frequency $\omega_0=\omega_{res}$:

$$
\tilde{F}(\omega) = \exp \left[ -\frac{1}{2} \sigma^2 \left( \omega - \omega_0 \right)^2 \right]
$$

This means that the effective promoting coefficient for edge frequency, 529 ev, will be: 

$$
\tau(529 \text{ev}) =  \left| \tilde{F}(529 \text{ev})  \times  \mathbf{\mu}^{529 \text{ev}} \cdot \mathbf{F} \right|^2 = \left| \mathbf{\mu}^{529 \text{ev}} \cdot \mathbf{F} \right|^2 
$$

while the one for the next available state will be smaller due to the relative frequency amplitude:

$$
\tau(535.5 \text{ev}) =  \left| \tilde{F}(535.5 \text{ev})  \times  \mathbf{\mu}^{535.5 \text{ev}} \cdot \mathbf{F} \right|^2 
$$

In this case: 

$$
\tilde{F}(535.5 \text{ev}) =  \exp \left[ -\frac{1}{2} \sigma^2 \left( 6.5 \text{ev} \right)^2 \right]
$$

The larger $\sigma$, the smaller $\tilde{F}(535.5 \text{ev})$ and thus $\tau(535.5 \text{ev})$. 
It means that the Real-Time simulation will promote electrons only toward the first available state, and not the second one.

In general, let us assume that we have our targetted state at $\omega_{res}$, and a second available one at $\omega_{res} + \omega_{e}$ 
We want the amplitude of the pulse in the frequency domain to be $Y$ times smaller than the maximal value at a frequency of the second available state: $\tilde{F}(\omega_{res} + \omega_e) = Y$.
Doing so, the ratio between the effective promoting coefficient and the total number of promoted electrons will be:

$$
\frac{\tau(\omega_{res})}{\tau(\omega_{res}+\omega_e)} = \frac{ \left| \tilde{F}(\omega_{res})  \times  \mathbf{\mu}^{\omega_{res}} \cdot \mathbf{F} \right|^2}{\left| \tilde{F}(\omega_{res}+\omega_e)  \times  \mathbf{\mu}^{\omega_{res}+\omega_e} \cdot \mathbf{F} \right|^2} = \frac{1}{Y^2} \frac{ \left| \mathbf{\mu}^{\omega_{res}} \cdot \mathbf{F} \right|^2}{\left|\mathbf{\mu}^{\omega_{res}+\omega_e} \cdot \mathbf{F} \right|^2} 
$$

Therefore, imposing a small $Y$ leads to a decrease of the second state electronic promoting rate: the simulation is specific only with respect to the first one at the frequency $\omega_{res}$.

For a Gaussian shape, the equation the field parameters must fulfill are: 

$$
Y =  \exp \left[ -\frac{1}{2} \sigma^2 \left( \omega - \omega_e \right)^2 \right]
$$

Note that it would also work if the second state is at $\omega_{res} - \omega_e$ for a Gaussian envelope. 

You should set the value of Y and we depending on your system: the oscillator strength of the state around the targetted one really matters.
For instance, for carbon-monoxide Oxygen K-edge, we would like to not excite the second available state located 6.5 ev away from the first excitation. 
Because the targetted state is much brighter than the neighboring one (oscillator strength of 0.044 and 0.0008 respectively), a field amplitude 0.01 smaller at the second state frequency would already be enough.
This would mean Y=0.01 and $\omega_e$ = 6.5 ev.

The value of the Gaussian extension in the time domain is given by sigma: 

$$
\sigma = \frac{\sqrt{- 2  \ln Y }}{\omega_e} 
$$

In the carbon monoxide case, this would mean $\sigma$ = 0.3 fs.


In [ ]:
# Your parameters:
omega_e = 6.5 # in ev
Y  = 0.01 # from 0 to 1

# Output:
omega_e_fs = omega_e/(au_to_ev*au_to_fs) # in fs
sigma_opt = np.sqrt(- 2*np.log(Y))/(omega_e_fs) # in fs
print('sigma in the time domain for the gaussian=', sigma_opt, ' fs')

### Position of the Gaussian envelope in time: accuracy Vs efficiency

Once the width of the pulse in the time domain has been defined, we should define its position in time controlled by $T_0$. 
We would like at the beginning of the simulation the field amplitude is the smallest possible. 
Indeed, we start the calculation from a zero-field situation and applying a high field between t=$0^{-}$ and t=$0^{+}$ is (i) unphysical and (ii) can lead to crashes or worst, garbage results. 


The relative amplitude of the field at time t=$0^{+}$ is given by:

$$
U = \exp \left[ -\frac{1}{2} \left( \frac{T_0}{\sigma} \right)^2 \right] 
$$

Here we have neglected the phase used in the cos function: what matters here is the shape of the envelope. 

Thus, to have  a relative error U of a given value, for instance, $10^{-4}$, then the position of the pulse must be:

$$
T_0 = \sigma  \sqrt{- 2  \ln U }
$$

For instance, $T_0$ would be 1.3 fs in the carbon monoxide case. 
The time-dependent field thus expends within 2 x $T_0$ (after these values, the amplitude is below $10^{-4}$).
If one increases the allowed error (U), then the total time of the simulation can be decreased. 
Please keep in mind that the more intense the field, the less error could be allowed.

**The simulation may crash if the field amplitude at the very beginning of the simulation is too high!**



The simulation total time is often defined to be: 

$$
T_{\text{simulation}} = 2 T_0 + T_{\text{relax}}
$$

where $T_{\text{relax}}$ is some time after the ending to see the dynamics occurring on your system after the pulse excitation, typically a few femtoseconds. 

In [ ]:
# Your parameters:
U = 10**(-4) # from 0 to 1
T_relax = 1 # in fs 

# Output:
T_0_opt = sigma_opt*np.sqrt(-2*np.log(U))
T_total =  2*T_0_opt + T_relax
N_time = int(T_total/delta_t) + 1

print('T_0 is ', T_0_opt, ' fs')
print('The total time of the simulation will be:', T_total, ' in fs')
print('The number of time step needed is:', N_time)

### Summary of the field shape parameters

We have almost found all the parameters needed for the Real-Time Propagation calculation. 
The remaining one is the field amplitude absolute value. 

Before tackling this last issue, below are given some cells to help you find your own parameters and to look at the pulse in the time and frequency domains.


In [ ]:
# Time step
# Your parameters:
E = 1 # here it does not matter, please leave it to 1.
omega_ev = 529 #the frequency of the targetted state in ev
phi = 0*np.pi/180 # phase of the pulse. 0 means +cos, -90 means +sin.
N_ts_per_period = 10 # how many time step per period, try between 5-10. 


# Output
omega_fs = omega_ev/(au_to_ev*au_to_fs)
delta_t = 2*np.pi/(omega_fs*N_ts_per_period) #fs

print('The frequency for the fast oscillation in nm is:', (529/omega_ev)*from_ev_to_nm_at_529, ' nm')
print('The time step value is:', delta_t, ' fs')


In [ ]:
# Gaussian width
# Your parameters:
omega_e = 6.5 # in ev
Y  = 0.01 # from 0 to 1

# Output:
omega_e_fs = omega_e/(au_to_ev*au_to_fs)
sigma_opt = np.sqrt(- 2*np.log(Y))/(omega_e_fs)
print('sigma in the time domain for the gaussian=', sigma_opt, ' fs')

In [ ]:
# Total simulation time
# Your parameters:
U = 10**(-4) # from 0 to 1
T_relax = 1 # in fs 

# Output:
T_0_opt = sigma_opt*np.sqrt(-2*np.log(U))
T_total =  2*T_0_opt + T_relax
N_time = int(T_total/delta_t) + 1

print('T_0 is ', T_0_opt, ' in fs')
print('The total time of the simulation will be:', T_total, ' fs')
print('The number of time step needed is:', N_time)

In [ ]:
# The field in the time domain:
T_0 = T_0_opt
sig = sigma_opt

L_time = np.array([delta_t*k for k in range(0, N_time, 1)])

L_field_time =  gaussian_cos(L_time, omega_fs, phi, E, T_0, sig)

plt.figure(1, figsize=(10, 7))
plt.plot(L_time,  L_field_time, lw=3, label=r'$F(t)$')
plt.xlabel(r'Time [fs]' , fontsize=25)
plt.ylabel(r'Applied Field', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

plt.figure(2, figsize=(10, 7))
plt.plot(L_time,  L_field_time/E, 'o-', lw=3, label=r'$E_{env}$')
plt.plot(L_time,  L_field_time*0 + U , '--k', label=r'U=' + str(U), lw=3)
plt.plot(L_time,  L_field_time*0 - U , '--k', lw=3)
plt.xlabel(r'Time [fs]' , fontsize=25)
plt.ylabel(r'Applied Field', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.xlim([0., 0.1])
plt.ylim([-2*U, 2*U])
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


In [ ]:
# Performing Fourier transform: you may not change any parameters here
omega_min = omega_ev-(omega_e*1.5) # in ev
omega_max = omega_ev+(omega_e*1.5) # in ev
N_omega = 100 #nbr of frequencies, note that this nbr may be reduced if the total time of the simulation is too small

omega_precision = 2

L_damp = np.zeros(len(L_time)) + 1
        
L_omega, L_ft = ft_utils.perform_ft(L_time/au_to_fs, L_field_time, L_damp, omega_min/au_to_ev, omega_max/au_to_ev, N_omega, omega_precision=omega_precision)
        
L_omega_ev = L_omega*au_to_ev

L_field_tf_analytic = tf_gaussian_cos(L_omega/au_to_fs, omega_fs, phi, E, T_0, sig)   
  

In [ ]:
# Not that the quality of the analytical / computed comparison may be impacted by the time step value and the amplitude at time 0
to_plot = L_ft/np.max(np.absolute(L_ft))
to_plot_analytic = L_field_tf_analytic/np.max(np.absolute(L_field_tf_analytic))

plt.figure(1, figsize=(10, 7))
plt.plot(L_omega_ev, to_plot.real, lw=3, label='Computed')
plt.plot(L_omega_ev, to_plot_analytic.real, '--', lw=3, label='Analytic')
plt.plot([omega_ev, omega_ev],  [0, 1], '--', color='gray', lw=3)
plt.plot(L_omega_ev,  L_omega_ev*0, '-k', lw=3)
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Re Field Normalized', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


plt.figure(2, figsize=(10, 7))
plt.plot(L_omega_ev, to_plot.imag, lw=3, label='Computed')
plt.plot(L_omega_ev, to_plot_analytic.imag, '--', lw=3, label='Analytic')
plt.plot([omega_ev, omega_ev],  [0, 1], '--', color='gray', lw=3)
plt.plot(L_omega_ev,  L_omega_ev*0, '-k', lw=3)
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Imag Field Normalized', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)



In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(L_omega_ev, np.absolute(to_plot), label='Computed', lw=3)
plt.plot(L_omega_ev, np.absolute(to_plot_analytic), '--', label='Analytic', lw=3)
plt.plot([omega_ev, omega_ev],  [0, 1], '--', color='gray', lw=3)
plt.plot(omega_ev-omega_e,  Y, 'or', ms=15)
plt.plot(omega_ev+omega_e,  Y, 'or', ms=15)
plt.plot(L_omega_ev,  L_omega_ev*0, '-k', lw=3)
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'$\tilde{F}$', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

plt.figure(2, figsize=(10, 7))
plt.plot(L_omega_ev, np.absolute(to_plot), lw=3)
plt.plot([omega_ev, omega_ev],  [0, 1], '--', color='gray', lw=3)
plt.plot(omega_ev-omega_e,  Y, 'or', ms=15)
plt.plot(omega_ev+omega_e,  Y, 'or', ms=15)
plt.plot(L_omega_ev,  L_omega_ev*0 + Y, '--r', label='Y=' + str(Y), lw=3)
plt.plot(L_omega_ev,  L_omega_ev*0, '-k', lw=3)
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'$\tilde{F}$', fontsize=25)
plt.ylim([Y*0.5, Y*1.5])
plt.xticks([omega_ev-omega_e, omega_ev, omega_ev+omega_e], [r'$\omega_0 - \omega_e$', r'$\omega_0$', r'$\omega_0 + \omega_e$'], fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


### Which amplitude for the field: how many electrons do we want to promote?

Now that we have determined the field shape, let's find out which reasonable amplitude one should try.
Please note that this part has been written in the general mindset of describing the electronic dynamics occurring after an X-Ray absorption: therefore the goal is clearly to promote core electron with the incident time-dependent electrogmatic field. 

The excited state population $\rho^{\text{exc}}$ assuming that we start from the ground state is:

$$
\rho^{\text{exc}}(t) = \sin  \left( \frac{ | \mathbf{\mu}^{res} \cdot \mathbf{F}| \times A(t)}{2} \right)^2 = \sin  \left( \frac{  A(t) \times \sqrt{\tau}}{2} \right)^2  \approx \tau \left( \frac{A(t)}{2} \right)^2  \\
A(t) = \int_{- \infty}^t E_{env}(t') dt'
$$

where somehow $\mathbf{A}$ is the equivalent of the vector potential but for the amplitude of the electric field.

The evolution of the excited population is plotted below, according to this approximation, for carbon-monoxide oxygen K-edge (oscillator strength is 0.0440 a.u.).

Note that we assume that we are exactly at resonance and the field polarization is along $ \mathbf{\mu}^{res}$.



In [ ]:
E = 1 # here one can tune this value


# Please note that the frequency has to be converted into atomic unit: 
tau = 3/2*0.044017/(528.982511/au_to_ev) # 3/2 * oscillator strength / omega_res 
print(r'$\tau$ = ', tau)

L_envelope, L_A = compute_envelope_vec_pot(L_time, delta_t, E, T_0, sig)
L_rho_exc = area_theorem_popu(tau, L_A)
rho_exc_final = L_rho_exc[-1]


In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(L_time, L_envelope, lw=3, label='Field envelope')
plt.xlabel(r'Time [fs]' , fontsize=25)
plt.ylabel(r'$E_{env}$', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

plt.figure(2, figsize=(10, 7))
plt.plot(L_time, L_A, lw=3, label='Envelope vec pot')
plt.xlabel(r'Time [fs]' , fontsize=25)
plt.ylabel(r"$\int_{- \infty}^t E_{env}(t') dt'$", fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


plt.figure(3, figsize=(10, 7))
plt.plot(L_time, L_rho_exc, lw=3, label='Excited population')
plt.plot(L_time, L_time*0 + rho_exc_final, '--k', lw=3)
plt.xlabel(r'Time [fs]' , fontsize=25)
plt.ylabel(r'$\rho_{exc}$', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


In this resonant regime, the **integral** of the field envelope determines how many electrons are promoted.
Thus, both the amplitude of the field and the amount of time where it is applied are important.
Once the pulse ends, the excited state population is given by:

$$
\lim_{t \to + \infty} \rho_{exc}(t) =  \rho^{exc}_{\text{end}} = \sin^2 \left( \frac{A_{\text{end}} \times \sqrt{\tau}}{2} \right) \\
A_{\text{end}} =  \int_{- \infty}^{+ \infty}E_{env}(t') dt' =  \int_{- \infty}^{+ \infty} E_{env}(t') dt' = \int_{- \infty}^{+ \infty} E \, \exp \left[ - \frac{1}{2} \left( \frac{T_0-t'}{\sigma} \right)^2 \right] dt' = E \, \sigma \sqrt{2 \pi}
$$

So that the final amount of promoted electron  $\rho^{exc}_{\text{end}}$ is:

$$
\rho^{exc}_{\text{end}} = \sin^2 \left( \frac{\sqrt{\tau} \, E \, \sigma \sqrt{2 \pi}}{2} \right) \approx  2 \pi \tau \left( \frac{\, E \, \sigma}{2} \right)^2
$$

**IMPORTANT: Spin in Real Time Propagation**

Please note that the oscillator strength obtained using the Linear response TD-DFT assumes a promotion of 2 electrons in most cases. 
Hence, the oscillator strength should be understood as a rate for 2 electrons.

Yet, the area theorem in its sinus formulation describes a promotion of only one electron: $\sin^2$ maximum is 1.
Therefore, if the 2 spins are undistinguishable, the excited population will be twice the one predicted using the area formula, but using the effective rate for 1 electron.
This is done inside the function defined at the beginning of this notebook. Note that if one uses the linearised form the two formulation converges.

Note that in LR-TDDFT we can account for spin-coupling effects but not in Real-Time TDDFT in 2023 CP2K implementation.
Hence, if you perform an RTP calculation from a closed shell system, both electronic spins will be promoted at the same time. 
If you start from an excited state where the two spins are not equivalent, then each spin will have its own dynamics and corresponding effective promotion rate.

**IMPORTANT: Spin in Real Time Propagation**



We have fixed the time width of the pulse $\sigma$ according to a targetted width in the frequency domain.
Hence, we can choose the field amplitude with respect to the amount of electrons we want to promote:

$$
E \approx \frac{ 2  }{ \sigma } \sqrt{\frac{\rho^{exc}_{\text{end}}}{2 \pi \tau}}
$$


For carbon monoxide at the Oxygen K-edge, the oscillator strength is 0.044.
We have found that the time width should be 0.3 fs.
Hence, promoting 0.001 electrons in the excited state would require a field amplitude of 0.034 a.u. or a maximal intensity of about 4.08 $10^{13}$ W.cm$^{-2}$. 

Note that using a large intensity may also lead to non-linear effects.
Hence, if one wants to avoid these effects, the time width of the pulse should be larger so that the maximal amplitude can be reduced. 


In [ ]:
tau = 3/2*0.044017/(528.982511/au_to_ev) # 3/2 * oscillator strength / omega_res 
rho_exc_final = 0.001 # the final population of electron required
sig_au= sigma_opt/au_to_fs # in atomic unit


# Output
E = 2/sig_au*np.sqrt(rho_exc_final/(2*np.pi*tau))
I_cp2k = 3.50944*10**16*E**2


print('In order to have a final population of ', rho_exc_final, ' excited electrons, the amplitude of the field should be E=', E, ' a.u.')
print(r'It corresponds to an intensity of I= %.2E W.cm^{-2} (for cp2k input)' % Decimal(str(I_cp2k)))

L_envelope, L_A = compute_envelope_vec_pot(L_time, delta_t, E, T_0, sig)
L_rho_exc = area_theorem_popu(tau, L_A)

plt.figure(1, figsize=(10, 7))
plt.plot(L_time, L_rho_exc, lw=3, label='Excited population')
plt.plot(L_time, L_time*0 + L_rho_exc[-1], '--k', lw=3)
plt.xlabel(r'Time [fs]' , fontsize=25)
plt.ylabel(r'$\rho_{exc}$', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


### Summary

Let us review the different parameters for an RTP run with explicit fields and how to choose them.

**Field Frequency** 

The field frequency should match the one of the electronic transition targetted. 
One can use Linear response-based approaches, for instance, the XAS_TDP module of CP2K for X-Ray frequencies. 
Note that you should perform the calculations with the same basis set and functional as the RTP run. 

**Field Polarization**

The field polarization should trigger the electronic transition targeted. 
One can rely on the oscillator strength decomposition or on a symmetry-based argument using the system structure. 
One can also rely on polarization-resolved experiments. 

If you do not know which polarization to apply, set up 3 calculations with orthogonal field polarization. 
It can be along the x,y, and z directions, but also along other ones as long as they form an orthogonal basis set. 

**Time Step**

The time step should be chosen so that the frequency of the field, *i.e.* the fasted-induced electronic dynamics, is correctly sampled. 
For instance, having 5-10 time steps per field period. 

**Field shape**

The shape of the pulse should be carefully chosen in regard to what you want to achieve. 
If one wants to trigger only one electronic transition, then the pulse should be long and smooth in the time domain so that the frequencies excited are narrowed around the targetted one. 
If the electronic transitions are largely separated in frequency or if the field polarization can be tuned to avoid some states, then the field pulse can be shorter in the time domain and so the total simulation time. 
In this tutorial, we have presented the calculations using a Gaussian envelope but other envelopes can be used, for instance, Lorentzians. 

**Field amplitude**

The field amplitude should also be chosen according to your purpose.

If one wants to simulate the absorption within the perturbative regime, then only a ''small'' number of electrons should be promoted to the targetted excited state -- for instance $10^{-3}$. 
Using the effective promoting rate $\tau$, determined from the oscillator strength of LR-TDDFT calculations, one can determine the field amplitude once the field shape is fixed. 
The results obtained in such a perturbative hypothesis are discussed below for isolated carbon monoxide. 

If one wants to describe what happens beyond the perturbative regime, for instance when a full electron is promoted, then the results obtained by LR-TDDFT calculations should be used as a first guess but not as quantitative prediction. 
For instance, one can set the field amplitude so that 1 electron should be promoted by the end of the simulation using the area theorem. 
Yet, it is very likely that not a full electron will be promoted by the end of the run as the perturbative regime often fails for real systems. 
In this case, one should try many field shapes and amplitude to tackle its specific system and question! 

Please keep in mind that increasing the field amplitude can lead to *non-linear* effects, such as higher harmonic response. These effects can happen at the same time as the loss of the perturbative regime, or independently! One can simulate for instance second harmonic generation using an intense but short pulse within the perturbative regime, and promote a full electron with a weak and long pulse within the linear regime. 




# Analysis of a resonant RTP simulation

The simulation of an isolated carbon monoxide molecule under a resonant Oxygen K-edge electromagnetic field has been performed in the length gauge. 
The results should be available in the 'Data' folders.

In the first part, we will open the different results using the home-made Python package 'cp2kRTPtools', available at https://github.com/glb96/cp2kRTPtools. 
Then, we will open the different raw results and discuss them. 
Finally, we will compare the results obtained by this Real-Time approach and the one predicted previously based on the LR framework. 

## Read CP2K results using cp2kRTPtools module

The following cell defines the field applied and the time step of the simulation. 

In [ ]:
# where is the directory with the results
dirname = './Data'

# the actual parameters used in RTP.inp 
delta_t = 0.00078 # in fs

omega_ev = 529

wavelength = 2.34374655955 
I= 4.08E+13 # in W.cm^-2
sigma = 0.3073 # fs 
T_0 = 1.3190 # fs
phi = 0

# do not change these lines
E = np.sqrt(I/(3.50944*10**16)) # field amplitude in atomic units

omega_ev = wavelength*529/from_ev_to_nm_at_529
omega_fs = omega_ev/(au_to_ev*au_to_fs) # frequency in fs^-1

print('Omega=', omega_ev, ' ev')

The following cell read:

    - The evolution of the total energy of the system
    - The applied electric field
    - The induced dipole moment

The time list is constructed thanks to the time step defined previously.

In [ ]:
# energy
filename = dirname  + '/RTP.out'
L_energy = open_output_files.read_energy(filename)[:]
L_delta_energy = L_energy - L_energy[0]

# applied field, in the Length Gauge it is an electric field
filename = dirname  + '/applied_field'
L_field = open_output_files.read_applied_field(filename)[:]

# dipole moment, the cp2k default unit is Debye: converted into atomic unit
filename = dirname + '/dipole'
L_dipole = open_output_files.read_dipole_moment(filename)[1:]/au_to_D

# simulated time in fs
L_time = np.array([k*delta_t for k in range(1, 1+len(L_field), 1)])# in fs


The excited population is not straightforward to get in the Real Time propagation approach. 
In CP2K, we propose two ways of obtaining this quantity thanks to the projection of the time-dependent Molecular Orbitals (TD-MO) $|\phi_i(t)>$. 
One can project these TD-MO over the excited state obtained from the XAS_TDP calculation $|\phi^{res}>$:

$$
\rho^{exc}(t) = \sum_i |<\phi^{res}|\phi_i(t)>|^2 
$$

The sum over the $i$ goes over the TD-MOs. In a spin-independent calculation, there are $N_e/2$ TD-MO for close shell systems -- $N_e$ being the number of electrons.
In this case, this projection has to be multiplied by two to get the actual amount of electrons promoted. 

Another possibility is to project the TD-MO over their initial state, $|\phi_j(t=0)>$, to define the ground state population. The excited population is the amount of electrons leaving this state: 

$$
\rho^{exc}(t) = N_e - \rho^{gs}(t), \ \ \ \ \rho^{gs}(t) = \sum_{i,j} |<\phi_j(t=0)|\phi_i(t)>|^2 
$$

The excited population is not straightforward to get in the Real Time propagation approach. 
In CP2K, we propose two ways of obtaining this quantity thanks to the projection of the time-dependent Molecular Orbitals (TD-MO) $|\phi_i(t)>$. 
One can project these TD-MO over the excited state obtained from the XAS_TDP calculation $|\phi^{res}>$:

$$
\rho^{exc}(t) = \sum_i |<\phi^{res}|\phi_i(t)>|^2 
$$
The first input section defines the projection over the ground state (REFERENCE_TYPE SCF).
If $m$ reference states are present in the state defined in REF_MO_FILE_NAME, then $m$ output file will be printed (REF_MO_INDEX -1 and SUM_ON_ALL_REF .FALSE.). 
In each of these files, the projection of all the $n$ TD-MOs is performed (TD_MO_INDEX -1) and saved independently (SUM_ON_ALL_TD .FALSE.). 

For carbon monoxide in the spin-independent approach, $m=7$ reference states are defined and $n=7$ TD-MOs are available. 
There are thus 7 files containing the time-dependent projection of the $i$ TD-MO state over the $j$ reference one:

$$
\rho^{gs}_{ji}(t) =  2 \times |<\phi_j(t=0)|\phi_i(t)>|^2 
$$

Note that the factor 2 is due to the fact that 2 electrons share the same TD-MOs in this spin-independent real-time calculation.
One can perform spin-dependent RTP, perform the projection of the 2 spins, and then sum the results -- leading to the exact same results here. 

The second and third sections define the projection over the two degenerate excited states provided by the XAS_TDP module (REFERENCE_TYPE XAS_TDP). 
One state is defined to be the reference one for each section and all the TD-MOs are projected to this state (TD_MO_INDEX -1). 
The projection of the TD-MOs is saved independently (SUM_ON_ALL_TD .FALSE.) so that each projection contains $i$ projection per time step:

$$
\rho^{exc}_{i}(t) =  2 \times |<\phi^{res}|\phi_i(t)>|^2 
$$

Due to the spatial symmetry of the excited states, the total excited population is the sum of the projection of these two degenerate states. 


In [ ]:
# population analysis: projection of the time-dependent wave-function over reference molecular orbitals
LL_projection = {}
# Over the GS
filename = dirname  + '/RTP-PROJ_MO-1-REF-' + str(1) + '.dat'
L_popu, L_phase = open_output_files.read_mo_projection(filename)
L_popu_gs = np.zeros(len(L_popu[:,0]))
for ref_mo in range(1, Ne//2+1, 1):
    filename = dirname  + '/RTP-PROJ_MO-1-REF-' + str(ref_mo) + '.dat'
    L_popu, L_phase = open_output_files.read_mo_projection(filename)
    LL_projection['GS', ref_mo] = L_popu
    L_popu_gs += np.sum(L_popu[:,0:Ne//2], axis=1)
    
# Over the excited state from TDP_XAS
filename = dirname  + '/RTP-PROJ_MO-1-REF-' + str(2) + '.dat'
L_popu, L_phase = open_output_files.read_mo_projection(filename)
L_popu_xas = np.zeros(len(L_popu[:,0]))
for ref_exc in range(2, 4, 1):
    filename = dirname  + '/RTP-PROJ_MO-' + str(ref_exc) + '-REF-1.dat'
    L_popu, L_phase = open_output_files.read_mo_projection(filename)
    LL_projection['EXC', ref_exc-1] = L_popu
    L_popu_xas += np.sum(L_popu[:,0:Ne//2], axis=1)
# multiply by 2 because of spin
L_popu_gs = 2*L_popu_gs
L_popu_xas = 2*L_popu_xas

## Plot CP2K outputs

First is plotted the time-dependent electric field applied to the system along with the expected one.

Please note that the weird shape at 1.2 fs is because the time step is not exactly a multiple of the period.

In [ ]:
L_expected_field = gaussian_cos(L_time, omega_fs, phi, E, T_0, sigma)

plt.figure(1, figsize=(10, 7))
plt.plot(L_time, L_field[:,0], lw=3, label='read')
plt.plot(L_time, L_expected_field, '--', lw=3, label='expected')
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.xlabel(r'Time [fs]' , fontsize=25)
plt.ylabel(r'$F_{x}$ [a.u.]', fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


plt.figure(2, figsize=(10, 7))
plt.plot(L_time, L_field[:,0], lw=3, label='read')
plt.plot(L_time, L_expected_field, '--', lw=3, label='expected')
plt.xlim([1.3, 1.4])
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.xlabel(r'Time [fs]' , fontsize=25)
plt.ylabel(r'$F_{x}$ [a.u.]', fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)






In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(L_time, L_delta_energy*au_to_ev, lw=3)
plt.xlabel(r'Time [fs]' , fontsize=25)
plt.ylabel(r'$\Delta$ E [ev]', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


The dipole moment is plotted along the x, y, and z directions. 
Each direction is interesting and underlines a different aspect of the simulation.

**Along the field: x**

The field is along the x-axis. 
Hence, an oscillation of the electrons at the same frequency is induced. 
This will create a dipole moment whose amplitude will grow with the number of excited electrons.

Therefore, the dipole moment along the x-axis oscillates at the field frequency and the amplitude follows the same shape as the amount of promoted electron (or similarly the total energy).

**Orthogonal to the field: y**

Due to the symmetry of the molecule, a perturbation along the x-axis cannot create a response along the y-axis in the dipolar approach: the xy symmetry is not broken due to an x-polarized field.

Thus, no dipole moment can be induced in the y direction. 

**Along the electronic delocalization: z**

The dipole moment along the z-axis presents no fast oscillation but a slow one which looks like the one of the total energy. 
It is because the excited state has not the same electronic distribution as the core state: we promote electrons from the oxygen 1s to a state delocalized across the whole molecule (have a look at LR/LR-xasat2_1s_singlet_w1_s1-1.cube). 
Hence, the dipole moment of the molecule in its excited state along the Carbon-Oxygen bond, z-axis, is not the same as the one of the ground state. 

Therefore, the time evolution of the $z$ dipole moment is due to the promotion of electrons through the simulation: it starts at the ground state value and evolves toward the excited one.


In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(L_time, L_dipole[:, 0], lw=3)
plt.xlabel(r'Time [fs]' , fontsize=25)
plt.ylabel(r'$\mu_{x}$ [a.u.]', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

plt.figure(2, figsize=(10, 7))
plt.plot(L_time, L_dipole[:, 1], lw=3)
plt.xlabel(r'Time [fs]' , fontsize=25)
plt.ylabel(r'$\mu_{y}$ [a.u.]', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

plt.figure(3, figsize=(10, 7))
plt.plot(L_time, L_dipole[:, 2], lw=3)
plt.xlabel(r'Time [fs]' , fontsize=25)
plt.ylabel(r'$\mu_{z}$ [a.u.]', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

The next cell presents the projection of the TD-MOs over the initial states. 
This projection looks quite diagonal as it should be: we are in the perturbative regime so that the electronic structure stays close to the ground state one:

$$
\rho^{gs}_{ji}(t) =  |<\phi_j(t=0)|\phi_i(t)>|^2 \approx \delta_{ij}
$$

In [ ]:
for ref_mo in range(1, Ne//2+1, 1):
    plt.figure(ref_mo, figsize=(10, 7))
    L_popu = LL_projection['GS', ref_mo]
    for i in range(len(L_popu[0,:])):
        plt.plot(L_popu[:, i], label=r'TD MO ' + str(i+1), lw=3)
    plt.xlabel(r'Time Step' , fontsize=25)
    plt.ylabel(r'$\rho^{gs}$ over GS ' + str(ref_mo), fontsize=25)
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)
    plt.legend(fontsize=20)
    plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


To observe the evolution of the projection we have subtracted the first time step: 

$$
\Delta \rho^{gs}_{ji}(t) =  |<\phi_j(t=0)|\phi_i(t)>|^2 -  |<\phi_j(t=0)|\phi_i(t=0)>|^2 
$$

This emphasizes how many electrons leave their initial state. 
We can see that the most dramatic evolution has taken place for the first TD-MO state, corresponding to the Oxygen 1s. 
Indeed, the projection of this state over its initial 1s state decreases as expected.


In [ ]:
for ref_mo in range(1, Ne//2+1, 1):
    plt.figure(ref_mo, figsize=(10, 7))
    L_popu = LL_projection['GS', ref_mo]
    for i in range(len(L_popu[0,:])):
        plt.plot(L_popu[:, i]-L_popu[0, i], label=r'TD MO ' + str(i+1), lw=3)
    plt.xlabel(r'Time Step' , fontsize=25)
    plt.ylabel(r'$\Delta \rho^{gs}$ over GS ' + str(ref_mo), fontsize=25)
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)
    plt.legend(fontsize=20)
    plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)



To understand where this state is evolving to, let's see what tells the projection over the predicted excited state. 
Below is plotted the projection over the two degenerate excited states provided by the XAS_TDP module. 

We can observe that the projection of all the states except the first one is zero. 
The initial 1s look more and more to these excited as the field is applied: we are promoting electrons by modifying the Molecular Orbital coefficient.


In [ ]:
trotter_figure = 1 
for ref_exc in range(2, 4, 1):
    plt.figure(ref_exc-1, figsize=(10, 7))
    L_popu = LL_projection['EXC', ref_exc-1]
    for i in range(len(L_popu[0,:])):
        plt.plot(L_popu[:, i]-L_popu[0, i], label=r'TD MO ' + str(i+1), lw=3)
    plt.xlabel(r'Time Step' , fontsize=25)
    plt.ylabel(r'$\rho^{exc}$ over XAS ' + str(ref_exc-1), fontsize=25)
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)
    plt.legend(fontsize=20)
    plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


Finally, the following figure shows the excited electronic number according to the projection over the ground state or the excited state provided by LR-TDDFT. 
The two definitions agree perfectly in this case: it means that we have indeed depopulated the core electron toward the targetted excited state. No other state has been populated. 

In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(Ne-L_popu_gs, label=r'$Ne-\rho^{gs}$', lw=3)
plt.plot(L_popu_xas, label=r'$\rho^{exc}$', lw=3)
plt.xlabel(r'Time Step' , fontsize=25)
plt.ylabel(r'$\rho$', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


## Comparison between LR and RTP

To conclude this tutorial, we can also compare the effective promotion rate $\tau$ obtained from the RTP approach and the one predicted by the XAS_TDP module.

Using the area theorem, we can compute a time-dependent $\tau$ which represents how many electrons have been promoted at time t.
If we are in the perturbative regime, this rate should be constant.

Note that we do not compute this rate at the beginning of the simulation because the field applied is too weak and the output would be very noisy.

In [ ]:
# calculation of the effective oscillator strength 
L_tau_exc_t = np.zeros(len(L_time))
L_tau_gs_t = np.zeros(len(L_time))
L_envelope, L_A = compute_envelope_vec_pot(L_time, delta_t, E, T_0, sigma) # L_A is in a.u.

for ttt in range(len(L_A)):
    if L_A[ttt]/L_A[-1] > 10**(-3):
        L_tau_exc_t[ttt] = computing_tau_from_rho_exc(L_popu_xas[ttt],L_A[ttt])
    
for ttt in range(len(L_A)):
    if L_A[ttt]/L_A[-1] > 10**(-3) and (Ne-L_popu_gs[ttt])>0 :
        L_tau_gs_t[ttt] = computing_tau_from_rho_exc((Ne-L_popu_gs[ttt]),L_A[ttt])
    

The obtained time-dependent $\tau$ is plotted below. 
We have a good agreement between both projection methods and the LR-TDDFT prediction.

In [ ]:
tau_LR = 3/2*0.044017/(528.982511/au_to_ev) # 3/2 * oscillator strength / omega_res 

plt.figure(1, figsize=(10, 7))
plt.plot(L_time, L_time*0 + tau_LR, '--k', label='LR-TDDFT', lw=3)
plt.plot(L_time, L_tau_exc_t, label='RTP/GS-based', lw=3)
plt.plot(L_time, L_tau_gs_t, label='RTP/Exc-based', lw=3)
plt.xlabel(r'Time [fs]' , fontsize=25)
plt.ylabel(r'$\tau$ [a.u.]', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)



If one wants to be picky, we can argue that we were not exactly at resonance according to the TD-TDDFT calculation. Hence, the effective promotion rate should be modified by the relative frequency amplitude $\tilde{F}(\omega_{exc})$: 

$$
| \tilde{F}(\omega) | \propto \, \exp \left[ -\frac{1}{2} \sigma^2 \left( \omega - \omega_0 \right)^2 \right] \\
\tau(\omega_{res}) =  \left| \tilde{F}(\omega_{res})  \times  \mathbf{\mu}^{res} \cdot \mathbf{F} \right|^2 
$$

Yet, this does not change the results.

In [ ]:
# Compute the relative frequency amplitude at the resonant frequency:
omega_res = 528.982511 # according to XAS_TDP in ev 
omega_calculation = omega_ev # the actual field frequency in ev

delta_omega_in_au = (omega_calculation-omega_res)/au_to_ev # in au
sigma_in_au = sigma/au_to_fs # in au 
F_omega_res = np.exp(-1/2*(sigma_in_au*delta_omega_in_au)**2) # relative weight for gaussian field envelope
print('The relative amplitude exactly at the resonant frequency is:', F_omega_res)

In [ ]:
tau_LR = 3/2*0.044017/(528.982511/au_to_ev) # 3/2 * oscillator strength / omega_res 

plt.figure(1, figsize=(10, 7))
plt.plot(L_time, L_time*0 + tau_LR*(1**2), '--k', label='LR-TDDFT', lw=3)
plt.plot(L_time, L_tau_exc_t*(F_omega_res**2), label='RTP/GS-based', lw=3)
plt.plot(L_time, L_tau_gs_t*(F_omega_res**2), label='RTP/Exc-based', lw=3)
plt.xlabel(r'Time [fs]' , fontsize=25)
plt.ylabel(r'$\tau(\omega_{res})$ [a.u.]', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

As a concluding plot, we can give an interpretation of the total energy thanks to the excited population: 

$$
\Delta E(t) = \hbar \omega_{res} \rho^{exc}(t)
$$

In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(L_time, L_delta_energy*au_to_ev, 'k-', label=r'$\Delta E$', lw=3)
plt.plot(L_time, (Ne-L_popu_gs)*omega_ev, 'r--', label=r'$\hbar \omega \times (Ne - \rho_{gs})$', lw=3)
plt.plot(L_time, L_popu_xas*omega_ev, 'b--', label=r'$\hbar \omega \times \rho_{exc}$', lw=3)    

plt.xlabel(r'Time [fs]' , fontsize=25)
plt.ylabel(r'$\Delta E$ [ev]', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)
    
